In [39]:
import yfinance as yf
import pandas as pd

alpha_data = pd.read_csv('combined_alpha.csv')
index_data = pd.read_csv('total_index_ticker_marketcap.csv')

START_DATE = '2024-06-03'
END_DATE = '2024-11-19'

In [40]:
midcap_index = pd.read_csv('data/historic_data/Nifty_Midcap_150.csv')
smallcap_index = pd.read_csv('data/historic_data/Nifty_Smallcap_250.csv')
midcap_index['Price'] = midcap_index['Price'].replace(',', '', regex=True).astype(float)
smallcap_index['Price'] = smallcap_index['Price'].replace(',', '', regex=True).astype(float)

nifty50 = yf.Ticker('^NSEI').history(start=START_DATE,end=END_DATE)
nifty50_return = ((nifty50['Close'].iloc[-1] - nifty50['Close'].iloc[0])/ nifty50['Close'].iloc[0]) * 100

midcap_start = float(midcap_index[midcap_index['Date']=='03-06-2024']['Price'].iloc[0])
midcap_end = float(midcap_index.iloc[0]['Price'])
midcap150_return =round (((midcap_end-midcap_start)/midcap_start) * 100,2)

smallcap_start = float(smallcap_index[smallcap_index['Date']=='03-06-2024']['Price'].iloc[0])
smallcap_end = float(midcap_index.iloc[0]['Price'])
smallcap250_return = round(((smallcap_end-smallcap_start)/smallcap_start)*100,2)


In [41]:
data = pd.merge(alpha_data,index_data[['Ticker','Market Cap']],on='Ticker',how='left')
print(data)

                          Company Name         Ticker      NSE Symbol  \
0                       ABB India Ltd.         ABB.NS         NSE:ABB   
1                     Adani Power Ltd.  ADANIPOWER.NS  NSE:ADANIPOWER   
2        Aditya Birla Real Estate Ltd.       ABREL.NS       NSE:ABREL   
3    Amara Raja Energy & Mobility Ltd.       ARE&M.NS       NSE:ARE&M   
4                 Apar Industries Ltd.    APARINDS.NS    NSE:APARINDS   
..                                 ...            ...             ...   
61                        Siemens Ltd.     SIEMENS.NS     NSE:SIEMENS   
62  Sun Pharmaceutical Industries Ltd.   SUNPHARMA.NS   NSE:SUNPHARMA   
63                    Tata Motors Ltd.  TATAMOTORS.NS  NSE:TATAMOTORS   
64        Torrent Pharmaceuticals Ltd.  TORNTPHARM.NS  NSE:TORNTPHARM   
65                        Vedanta Ltd.        VEDL.NS        NSE:VEDL   

   Market Cap  
0   Large Cap  
1   Large Cap  
2   Small Cap  
3   Small Cap  
4   Small Cap  
..        ...  
61  Large C

In [42]:
def calulate_return(ticker,period):
    stock = yf.Ticker(ticker).history(period=period)

    if not stock.empty:
        start_price = stock['Close'].iloc[0]
        last_price = stock['Close'].iloc[-1]
        returns = round(((last_price-start_price)/start_price)*100, 2)
        print(f'{ticker} has given {returns} return in last {period}')
        return float(returns)

    else:
        print(f'{ticker} does not have enough data for given period {period}')
        return None

In [43]:
def calculate_absolute_momentum(ticker, start_date=None, end_date=None, period=None):

    if (start_date is None or end_date is None) and period is None:
        print('Provide either period or both start date and end date')
        return None

    try:
        if period:
            stock = yf.Ticker(ticker).history(period=period)
        else:
            stock = yf.Ticker(ticker).history(start=start_date, end=end_date)

        if not stock.empty:
            start_price = stock['Close'].iloc[0]
            last_price = stock['Close'].iloc[-1]
            absolute_momentum_score = last_price/start_price
            print(f'{ticker} has absolute momentum score of {
                  absolute_momentum_score}')
            return round(float(absolute_momentum_score), 4)
        else:
            print(f'{ticker} does not have enough data')
            return None

    except Exception as e:
        print(f"Error calculating absolute momentum: {str(e)}")
        return None


def calculate_relative_momentum(ticker, market_cap, start_date=None, end_date=None, period=None):
    if (start_date is None or end_date is None) and period is None:
        print('Provide either period or both start date and end date')
        return None

    market_cap_ticker = {
        'Large Cap': nifty50_return,
        'Mid Cap': midcap150_return,
        'Small Cap': smallcap250_return,
    }

    if market_cap not in market_cap_ticker:
        print("Invalid market cap category. Choose from: 'Large Cap', 'Mid Cap', 'Small Cap', 'Micro Cap'.")
        return None

    try:

        if period:
            stock_data = yf.Ticker(ticker).history(period=period)
        else:
            stock_data = yf.Ticker(ticker).history(
                start=start_date, end=end_date)
            

        if stock_data.empty:
            print(f"Data is not available for {ticker} in the given period.")
            return None

        stock_start_price = stock_data['Close'].iloc[0]
        stock_end_price = stock_data['Close'].iloc[-1]
        stock_return = ((stock_end_price - stock_start_price) /
                        stock_start_price) * 100

        index_return = market_cap_ticker[market_cap]

        if index_return == 0:
            print("Warning: Index return is zero, cannot calculate relative momentum")
            return None

        relative_momentum = stock_return/index_return
        print(f'{ticker} has relative momentum score of {relative_momentum}')
        return round(float(relative_momentum), 4)

    except Exception as e:
        print(f"Error calculating relative momentum: {str(e)}")
        return None

def calculate_dual_momentum(ticker, market_cap, start_date=None, end_date=None, period=None,
                            absolute_momentum_weightage=50, relative_momentum_weightage=50):
    if absolute_momentum_weightage + relative_momentum_weightage != 100:
        print(f"Error: The weightages must sum to 100. Given: {
              absolute_momentum_weightage + relative_momentum_weightage}")
        return None

    try:
        relative_momentum = calculate_relative_momentum(
            ticker, market_cap, start_date, end_date, period)
        absolute_momentum = calculate_absolute_momentum(
            ticker, start_date, end_date, period)

        if relative_momentum is None or absolute_momentum is None:
            return None

        dual_momentum = (absolute_momentum * absolute_momentum_weightage) / 100 + \
            (relative_momentum * relative_momentum_weightage) / 100

        print(f'{ticker} has a dual momentum score of {dual_momentum}')
        return round(dual_momentum, 4)

    except Exception as e:
        print(f"Error calculating dual momentum: {str(e)}")
        return None


In [44]:
def df_relative_momentum(row):
    return calculate_relative_momentum(ticker=row['Ticker'],start_date=START_DATE,end_date=END_DATE,market_cap=row['Market Cap'])

def df_absolute_momentum(row):
    return calculate_absolute_momentum(ticker=row['Ticker'],start_date=START_DATE,end_date=END_DATE)

def df_dual_momentum(row):
    return calculate_dual_momentum(ticker=row['Ticker'],start_date=START_DATE,end_date=END_DATE,market_cap=row['Market Cap'])


In [45]:
data['Relative Momentum'] = data.apply(df_relative_momentum,axis=1)
data['Absolute Momentum'] = data.apply(df_absolute_momentum,axis=1)
data['Dual Momentum'] = data.apply(df_dual_momentum,axis=1)

ABB.NS has relative momentum score of -29.378670676564788
ADANIPOWER.NS has relative momentum score of -48.70113940009223
ABREL.NS has relative momentum score of 0.7796727132431704
ARE&M.NS has relative momentum score of 0.11815974902957456
APARINDS.NS has relative momentum score of 0.4075298355448038
AUROPHARMA.NS has relative momentum score of 0.43164519632157283
BSE.NS has relative momentum score of 49.50052034027351
BAJAJ-AUTO.NS has relative momentum score of 3.3907953302419713
BDL.NS has relative momentum score of -27.218255136765023
BEL.NS has relative momentum score of -15.319229960351826
BHEL.NS has relative momentum score of -34.84717156387964
CDSL.NS has relative momentum score of 1.5305856273232776
COCHINSHIP.NS has relative momentum score of -23.35685101717874
COLPAL.NS has relative momentum score of 2.077445660816433
CUMMINSIND.NS has relative momentum score of -6.100206990744205
DIXON.NS has relative momentum score of 33.0304921157537
GLENMARK.NS has relative momentum sc

In [46]:
data

,Company Name,Ticker,NSE Symbol,Market Cap,Relative Momentum,Absolute Momentum,Dual Momentum
0,ABB India Ltd.,ABB.NS,NSE:ABB,Large Cap,-29.3787,0.7602,-14.3092
1,Adani Power Ltd.,ADANIPOWER.NS,NSE:ADANIPOWER,Large Cap,-48.7011,0.6025,-24.0493
2,Aditya Birla Real Estate Ltd.,ABREL.NS,NSE:ABREL,Small Cap,0.7797,1.2073,0.9935
3,Amara Raja Energy & Mobility Ltd.,ARE&M.NS,NSE:ARE&M,Small Cap,0.1182,1.0314,0.5748
4,Apar Industries Ltd.,APARINDS.NS,NSE:APARINDS,Small Cap,0.4075,1.1084,0.7580
...,...,...,...,...,...,...,...
61,Siemens Ltd.,SIEMENS.NS,NSE:SIEMENS,Large Cap,-10.6742,0.9129,-4.8807
62,Sun Pharmaceutical Industries Ltd.,SUNPHARMA.NS,NSE:SUNPHARMA,Large Cap,25.3132,1.2066,13.2599
63,Tata Motors Ltd.,TATAMOTORS.NS,NSE:TATAMOTORS,Large Cap,-22.7067,0.8146,-10.9461
64,Torrent Pharmaceuticals Ltd.,TORNTPHARM.NS,NSE:TORNTPHARM,Large Cap,19.8622,1.1621,10.5122


In [47]:
data.to_csv('momentum_stocks.csv',index=False)